In [6]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from lxml import etree
import requests
import time
import re
import pandas as pd

# 由于默认的Edge的webdriver和我下的不太一样，所以这里要指定路径
driver_path = "F:/Python/msedgedriver.exe"
driver = webdriver.Edge(executable_path=driver_path) # 指定driver路径
wait = WebDriverWait(driver, 10) # 等待10秒

# 短评的网址
url = "https://movie.douban.com/subject/26266893/comments?status=P"

driver.get(url) # 浏览器打开页面


# 下一页的按钮
next_page = wait.until(EC.element_to_be_clickable(
    (By.XPATH, '//*[@id="paginator"]/a') # 使用XPath提取
)) # 直到元素可被点击
print(next_page)

<selenium.webdriver.remote.webelement.WebElement (session="7c9ddc456cfff2d324522ce0088a0c29", element="c1d9bbee-995b-47c1-bbeb-19ab1f590a5d")>


In [9]:
# 网页源码解析，得到dom文件
dom = etree.HTML(driver.page_source, etree.HTMLParser(encoding='utf-8')) 

In [55]:
# 提取用户名
names = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/a/text()')
print(names, len(names))

# 提取用户主页的地址
user_urls = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/a/@href')
print(user_urls, len(user_urls))

# 提取评分
ratings = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/span[2]/@class')
print(ratings, len(ratings))

# 提取时间 
# 这里是需要用last()因为有的用户没有评分，这样就会少一个span
# 为了后边保证数据都是20条且后边剔除，这里采用的last()
times = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/span[last()]/@title')
print(times, len(times))

# 提取有用数量
votes = dom.xpath('//div[@class="comment-item"]//div[@class="comment"]//span[@class="votes"]/text()')
print(votes, len(votes))

# 提取短评正文
messages = dom.xpath('//div[@class="comment-item"]//div[@class="comment"]/p/span[@class="short"]/text()')
print(messages, len(messages))

['陆支羽', '影志', 'gus', '乌鸦火堂', '张小北', '舒服Kimodi', '沙雕电影', 'frozenmoon', '姨妈的鸭', '[已注销]', '凌睿', '亵渎电影', '江寒园', '桃桃林林', '刘小流', '巽', '倒带', '九只苍蝇撞墙', '王小叶儿', '阿暖'] 20
['https://www.douban.com/people/luzhiyu/', 'https://www.douban.com/people/tjz230/', 'https://www.douban.com/people/168391468/', 'https://www.douban.com/people/dreamfox/', 'https://www.douban.com/people/xzfd/', 'https://www.douban.com/people/58713868/', 'https://www.douban.com/people/185573840/', 'https://www.douban.com/people/1233038/', 'https://www.douban.com/people/yimaxxduck/', 'https://www.douban.com/people/hans_berdych/', 'https://www.douban.com/people/lingrui1995/', 'https://www.douban.com/people/zhangzongqian/', 'https://www.douban.com/people/63306399/', 'https://www.douban.com/people/qijiuzhiyue/', 'https://www.douban.com/people/yingpingduli/', 'https://www.douban.com/people/nothingbut/', 'https://www.douban.com/people/1892627/', 'https://www.douban.com/people/huhu1999/', 'https://www.douban.com/people/wangxiaoyeer/', '

In [41]:
# cookies放在请求头里就可以了
headers = {
    'Cookie': 'bid="54hNxxWZfvI"; ll="118299"; douban-fav-remind=1; dbcl2="25506168:LfjTnVX2f5g"; ck=DArE; push_noty_num=0; push_doumail_num=0; ap_v=0,6.0',
    'User-Agent': 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36 Edg/83.0.478.61'
}

cities = []
join_times = []

# 遍历每个用户的主页地址 这里就有问题了，如果是已注销用户是没有主页的所以就不去解析他的主页了
for i in range(len(user_urls)):
    if(names[i] == "[已注销]"):
        cities.append("未知")
        join_times.append("未知")
        continue
    res = requests.get(user_urls[i], headers=headers)
    user_dom = etree.HTML(res.content, etree.HTMLParser(encoding='utf-8'))
    address = user_dom.xpath('//div[@class="user-info"]/a/text()') # 用户居住地 有可能没设置用户居住地
    join_time = user_dom.xpath('//div[@class="user-info"]//div[@class="pl"]/text()') # 加入时间 这里会匹配到两个值，第一个值是用户id第二个值才是加入时间
    if len(address) == 0:
        cities.append("未知")
    else:
        cities.append(address[0])
    join_times.append(join_time[1]) # 这时获取的的加入时间还是有  空格 和  加入两个字的，后面要进行数据处理
    print(f"用户 {names[i]} 主页爬取完毕")
    time.sleep(2) # 反正ip被封每隔2秒，才去爬下个用户

用户 陆支羽 主页爬取完毕
用户 影志 主页爬取完毕
用户 gus 主页爬取完毕
用户 乌鸦火堂 主页爬取完毕
用户 张小北 主页爬取完毕
用户 舒服Kimodi 主页爬取完毕
用户 沙雕电影 主页爬取完毕
用户 frozenmoon 主页爬取完毕
用户 姨妈的鸭 主页爬取完毕
用户 凌睿 主页爬取完毕
用户 亵渎电影 主页爬取完毕
用户 江寒园 主页爬取完毕
用户 桃桃林林 主页爬取完毕
用户 刘小流 主页爬取完毕
用户 巽 主页爬取完毕
用户 倒带 主页爬取完毕
用户 九只苍蝇撞墙 主页爬取完毕
用户 王小叶儿 主页爬取完毕
用户 阿暖 主页爬取完毕


In [42]:
print(cities, len(cities))
print(join_times, len(join_times))

['北京', '北京', '广东广州', '北京', '北京', '辽宁大连', '北京', '北京', '上海', '未知', '重庆', '北京', '陕西西安', "P'yŏngyang, North Korea", '上海', '未知', '江苏南京', 'Montréal, Canada', '北京', '北京'] 20
[' 2008-08-30加入', ' 2005-07-18加入', ' 2017-10-19加入', ' 2008-03-12加入', ' 2008-03-04加入', ' 2012-02-19加入', ' 2018-10-07加入', ' 2006-08-12加入', ' 2009-12-01加入', '未知', ' 2012-08-07加入', ' 2009-04-14加入', ' 2012-07-22加入', ' 2005-10-09加入', ' 2012-11-21加入', ' 2007-12-07加入', ' 2007-10-05加入', ' 2007-01-11加入', ' 2011-04-12加入', ' 2009-03-08加入'] 20


In [47]:
effective_ratings = ['' if 'rating' not in i else int(re.findall('\d{2}', i)[0]) for i in ratings] # 提取出评分数据的有效部分
print(effective_ratings, len(effective_ratings))

[50, 40, 20, 50, 50, 50, 40, 30, 50, 20, 40, 40, 30, 30, 40, 50, 20, '', 50, 30] 20


In [52]:
print(join_times)
effective_join_times = [i.strip()[:-2] if i != "未知" else "未知" for i in join_times] # 处理加入时间
print(effective_join_times, len(effective_join_times))

[' 2008-08-30加入', ' 2005-07-18加入', ' 2017-10-19加入', ' 2008-03-12加入', ' 2008-03-04加入', ' 2012-02-19加入', ' 2018-10-07加入', ' 2006-08-12加入', ' 2009-12-01加入', '未知', ' 2012-08-07加入', ' 2009-04-14加入', ' 2012-07-22加入', ' 2005-10-09加入', ' 2012-11-21加入', ' 2007-12-07加入', ' 2007-10-05加入', ' 2007-01-11加入', ' 2011-04-12加入', ' 2009-03-08加入']
['2008-08-30', '2005-07-18', '2017-10-19', '2008-03-12', '2008-03-04', '2012-02-19', '2018-10-07', '2006-08-12', '2009-12-01', '未知', '2012-08-07', '2009-04-14', '2012-07-22', '2005-10-09', '2012-11-21', '2007-12-07', '2007-10-05', '2007-01-11', '2011-04-12', '2009-03-08'] 20


In [57]:
data = pd.DataFrame({
    '用户名': names,
    '居住城市': cities,
    '加入时间': effective_join_times,
    '评分': effective_ratings,
    '发表时间': times,
    '短评正文': messages,
    '有用数量': votes
})
data

,用户名,居住城市,加入时间,评分,发表时间,短评正文,有用数量
0,陆支羽,北京,2008-08-30,50,2019-01-29 20:10:48,1.终于，轮到我们仰望星空。2.后启示录死亡废墟，赛博朋克地下城，以及烟波浩渺的末日想象，缔...,62234
1,影志,北京,2005-07-18,40,2019-02-04 15:56:16,“北京道路安全委提醒你：道路千万条，安全第一条，行车不规范，亲人两行泪” 这句广播语真是又土...,71725
2,gus,广东广州,2017-10-19,20,2019-02-05 01:33:03,台词做作剧情薄弱没一个演员在线 无时无刻的尬笑走心台词还以为编剧是激素泛滥的中年少女 感觉看...,14458
3,乌鸦火堂,北京,2008-03-12,50,2019-01-20 19:00:54,华语真正意义上的第一部科幻大片！刘慈欣的硬核科幻设定，小兵扛大旗的主流价值观人设，东方式的家...,15016
4,张小北,北京,2008-03-04,50,2019-01-29 02:11:13,从各个方面来说都是一部好看的类型片。而最特别的是它是一部国产科幻电影，影迷+科幻迷的双重满足...,12905
5,舒服Kimodi,辽宁大连,2012-02-19,50,2019-02-06 03:54:42,8.8，热评第一条我就呵呵了，中国科幻电影为什么不值得我们打高分去鼓励，非要放在国际水平线上...,9607
6,沙雕电影,北京,2018-10-07,40,2019-02-05 00:24:35,一个悲伤的故事：太阳都要毁灭，地球都要流浪了，我国的校服还是这么丑......,40972
7,frozenmoon,北京,2006-08-12,30,2019-02-05 16:29:48,1.从特效和技术上讲，这应该是迄今为止此类中国电影的巅峰了。磅礴恢宏，细节营造用心。2.故事...,4377
8,姨妈的鸭,上海,2009-12-01,50,2019-02-05 16:51:54,春节期间最大的一场烟花，是看你爸为你和地球点燃的宇宙爆炸，太朋克了。,6707
9,[已注销],未知,未知,20,2019-01-20 23:12:44,不能因为它是首部本土科幻片就蒙蔽了双眼打高分，对吧。看完要说的仍然是中国电影工业还有很长的路...,7946


# 将上面的代码整理一下，并加获取一页数据的代码放在一个函数内

In [58]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from lxml import etree
import requests
import time
import re
import pandas as pd

In [61]:
# 由于默认的Edge的webdriver和我下的不太一样，所以这里要指定路径
driver_path = "F:/Python/msedgedriver.exe"
driver = webdriver.Edge(executable_path=driver_path) # 指定driver路径
wait = WebDriverWait(driver, 10) # 等待10秒

# 短评的网址
url = "https://movie.douban.com/subject/26266893/comments?status=P"

driver.get(url) # 浏览器打开页面

# cookies放在请求头里
headers = {
    'Cookie': 'bid="54hNxxWZfvI"; ll="118299"; douban-fav-remind=1; dbcl2="25506168:LfjTnVX2f5g"; ck=DArE; push_noty_num=0; push_doumail_num=0; ap_v=0,6.0',
    'User-Agent': 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36 Edg/83.0.478.61'
}

In [86]:
# 整理成的获取每页数据的函数
def get_web_data(dom=None, headers=None):
    # 提取用户名
    names = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/a/text()')

    # 提取用户主页的地址
    user_urls = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/a/@href')

    # 提取评分
    ratings = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/span[2]/@class')

    # 提取时间 
    # 这里是需要用last()因为有的用户没有评分，这样就会少一个span
    # 为了后边保证数据都是20条且后边剔除，这里采用的last()
    times = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/span[last()]/@title')

    # 提取有用数量
    votes = dom.xpath('//div[@class="comment-item"]//div[@class="comment"]//span[@class="votes"]/text()')

    # 提取短评正文
    messages = dom.xpath('//div[@class="comment-item"]//div[@class="comment"]/p/span[@class="short"]/text()')
    
    cities = [] # 居住地
    join_times = [] # 加入时间

    # 遍历每个用户的主页地址 这里就有问题了，如果是已注销用户是没有主页的所以就不去解析他的主页了
    for i in range(len(user_urls)):
#         if(names[i] == "[已注销]"):
#             cities.append("未知")
#             join_times.append("未知")
#             continue
        res = requests.get(user_urls[i], headers=headers)
        user_dom = etree.HTML(res.content, etree.HTMLParser(encoding='utf-8'))
        address = user_dom.xpath('//div[@class="user-info"]/a/text()') # 用户居住地 有可能没设置用户居住地
        join_time = user_dom.xpath('//div[@class="user-info"]//div[@class="pl"]/text()') # 加入时间 这里会匹配到两个值，第一个值是用户id第二个值才是加入时间
        
        # 可能被封号了或者账号被注销了 那么用户的居住地和计入时间都无法获取到
        # 但是为了保证数据量一致，这里用未知代替
        if len(address) == 0: # 还可能未设置居住地
            cities.append("未知")
        else:
            cities.append(address[0])
        
        if len(join_time) == 0: 
            join_times.append("未知")
        else:
            join_times.append(join_time[1]) # 这时获取的的加入时间还是有  空格 和  加入两个字的，后面要进行数据处理

        print(f"\t用户 {names[i]} 爬取完毕")
        time.sleep(1) # 反正ip被封每隔1秒，才去爬下个用户
        
    # 提取出评分数据的有效部分
    effective_ratings = ['' if 'rating' not in i else int(re.findall('\d{2}', i)[0]) for i in ratings]
    # 处理加入时间
    effective_join_times = [i.strip()[:-2] if i != "未知" else "未知" for i in join_times]
    
    # 构建成DataFrame并返回
    data = pd.DataFrame({
        '用户名': names,
        '居住城市': cities,
        '加入时间': effective_join_times,
        '评分': effective_ratings,
        '发表时间': times,
        '短评正文': messages,
        '有用数量': votes
    })
    
    return data

In [88]:
all_data = pd.DataFrame()
n = 0
while True:
    # 确保当前页的最后一个用户按钮是否可以被点击
    # 即确保最后一个用户的短评被添加进网页后再执行下面的代码
    wait.until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, '#comments > div:nth-of-type(20) > div.comment > h3 > span.comment-info > a') # 使用CSS提取
    )) 
    print(f"开始爬取第 {n} 页...")
    
    # 网页源码解析，得到dom文件
    dom = etree.HTML(driver.page_source, etree.HTMLParser(encoding='utf-8'))
    data = get_web_data(dom=dom, headers=headers)
    all_data = pd.concat([all_data, data], axis=0) # 纵向拼接数据
    print("该页数据共：", data.shape)
    print("全部数据共：", all_data.shape)
    print(f"第 {n} 页爬取完毕！")
    n += 1
    
#     if driver.find_element_by_xpath('//*[@id="paginator"]/a[@class="next"]') == []: # 通过XPath查找下一页的按钮库存在
#         break # 这样不太对,会报错
    if n == 25: # 直接用n是否等于25作为终止条件，因为豆瓣的限制 只能爬取25页
        break
    
    # 获取下一页按钮
    next_page = wait.until(EC.element_to_be_clickable(
        (By.XPATH, '//*[@id="paginator"]/a[@class="next"]') # 使用XPath提取
    )) # 直到元素可被点击
    next_page.click() # 点击下一页 

开始爬取第 0 页...
	用户 影志 爬取完毕
	用户 说给自己听 爬取完毕
	用户 陆支羽 爬取完毕
	用户 沙雕电影 爬取完毕
	用户 乌鸦火堂 爬取完毕
	用户 张小北 爬取完毕
	用户 frozenmoon 爬取完毕
	用户 浅潜 爬取完毕
	用户 舒服Kimodi 爬取完毕
	用户 江寒园 爬取完毕
	用户 凌睿 爬取完毕
	用户 亵渎电影 爬取完毕
	用户 桃桃林林 爬取完毕
	用户 九只苍蝇撞墙 爬取完毕
	用户 刘小流 爬取完毕
	用户 阿暖 爬取完毕
	用户 公享史 爬取完毕
	用户 二十二岛主 爬取完毕
	用户 苏妄言 爬取完毕
	用户 姨妈的鸭 爬取完毕
该页数据共： (20, 7)
全部数据共： (20, 7)
第 0 页爬取完毕！
开始爬取第 1 页...
	用户 gus 爬取完毕
	用户 琦殿 爬取完毕
	用户 谢谢你们的鱼 爬取完毕
	用户 巽 爬取完毕
	用户 巴伐利亞酒神 爬取完毕
	用户 懒懒 爬取完毕
	用户 冰山的阴影 爬取完毕
	用户 寻～ 爬取完毕
	用户 [已注销] 爬取完毕
	用户 [已注销] 爬取完毕
	用户 谋杀游戏机 爬取完毕
	用户 小啊原 爬取完毕
	用户 倒带 爬取完毕
	用户 qw0aszx 爬取完毕
	用户 非常不小心 爬取完毕
	用户 BeAst 爬取完毕
	用户 Ou 爬取完毕
	用户 Seki怎么又瘦了 爬取完毕
	用户 木卫二 爬取完毕
	用户 王小叶儿 爬取完毕
该页数据共： (20, 7)
全部数据共： (40, 7)
第 1 页爬取完毕！
开始爬取第 2 页...
	用户 欢乐分裂 爬取完毕
	用户 喂饭 爬取完毕
	用户 shininglove 爬取完毕
	用户 同志亦凡人中文站 爬取完毕
	用户 孟三哥 爬取完毕
	用户 小姬 爬取完毕
	用户 汪金卫 爬取完毕
	用户 基闹 爬取完毕
	用户 Pekingcat 爬取完毕
	用户 信号无响应 爬取完毕
	用户 顿河 爬取完毕
	用户 春树 爬取完毕
	用户 Dyingpluto 爬取完毕
	用户 王大根 爬取完毕
	用户 后自愈 爬取完毕
	用户 Mr. Infamous 爬取完毕
	用户 芦哲峰 爬取完毕
	用户 LOOK 爬取完毕
	用户 变形的文科生 爬取完毕
	用户 彭萦 爬取完毕
该页数据共： (20, 7)
全部数据共：

	用户 易思棠 爬取完毕
	用户 巴喆 爬取完毕
	用户 宅蘑菇Moku 爬取完毕
	用户 刘二千 爬取完毕
	用户 舟行芳菲 爬取完毕
	用户 viennavirus 爬取完毕
	用户 姚璐 爬取完毕
	用户 Panda的阴影 爬取完毕
	用户 欧阳杼 爬取完毕
	用户 筷喜旺 爬取完毕
	用户 一只鲨鱼🦈 爬取完毕
	用户 肖恩恩恩恩肖 爬取完毕
该页数据共： (20, 7)
全部数据共： (480, 7)
第 23 页爬取完毕！
开始爬取第 24 页...
	用户 鱼二度 爬取完毕
	用户 Sue 爬取完毕
	用户 SleepWalker 爬取完毕
	用户 锦翼 爬取完毕
	用户 科学的兆民赖之 爬取完毕
	用户 妖孽坐地成仙 爬取完毕
	用户 无非 爬取完毕
	用户 梦魇马戏团 爬取完毕
	用户 月光博客 爬取完毕
	用户 唐那啥 爬取完毕
	用户 Jenny白老师 爬取完毕
	用户 舒农 爬取完毕
	用户 Neo 爬取完毕
	用户 大凤梨 爬取完毕
	用户 看守煙霧的人 爬取完毕
	用户 莫亚 爬取完毕
	用户 代号273 爬取完毕
	用户 周 老 爺 爬取完毕
	用户 王强尼乐百事 爬取完毕
	用户 拉妹 爬取完毕
该页数据共： (20, 7)
全部数据共： (500, 7)
第 24 页爬取完毕！


In [92]:
# 保存到本地 并且不用保存索引列
all_data.to_csv('liulangdiqiu.csv', encoding='GB18030', index=None)

In [91]:
all_data

,用户名,居住城市,加入时间,评分,发表时间,短评正文,有用数量
0,影志,北京,2005-07-18,40,2019-02-04 15:56:16,“北京道路安全委提醒你：道路千万条，安全第一条，行车不规范，亲人两行泪” 这句广播语真是又土...,71725
1,说给自己听,江苏南京,2012-07-29,10,2019-02-05 10:24:19,求求编剧，人类都快失去地球了，生存才是最重要的。请别再bb这点家庭琐事和煽情烂梗。白瞎了刘慈...,14280
2,陆支羽,北京,2008-08-30,50,2019-01-29 20:10:48,1.终于，轮到我们仰望星空。2.后启示录死亡废墟，赛博朋克地下城，以及烟波浩渺的末日想象，缔...,62235
3,沙雕电影,北京,2018-10-07,40,2019-02-05 00:24:35,一个悲伤的故事：太阳都要毁灭，地球都要流浪了，我国的校服还是这么丑......,40976
4,乌鸦火堂,北京,2008-03-12,50,2019-01-20 19:00:54,华语真正意义上的第一部科幻大片！刘慈欣的硬核科幻设定，小兵扛大旗的主流价值观人设，东方式的家...,15017
...,...,...,...,...,...,...,...
15,莫亚,北京,2016-10-04,50,2019-02-07 13:28:05,中国科幻电影有希望了，老泪纵横啊！就是演的太像“演的”了！两个小孩的戏，还有姥爷的配音有点出...,33
16,代号273,北京,2008-09-24,50,2019-02-05 10:52:11,看完以后，中国科幻电影元年不元年，中国科幻有没有希望，这种谁还在乎。仅说电影，已经完全值得吹...,22
17,周 老 爺,广东广州,2007-01-28,,2019-02-10 22:32:08,中国式煽情真是又臭又长,9
18,王强尼乐百事,浙江杭州,2010-07-27,20,2019-02-08 23:48:55,全片就很尬。剧情发展和突然的煽情，整个尬到最后；人物设定也很奇怪，全片找不到任何一个爆满的人...,39


In [93]:
driver.close() # 关闭浏览器